# Install libraries

In [ ]:
pip install pycaret

# Import packages

In [2]:
import numpy as np
import pandas as pd
from pycaret.time_series import *

# Read dataset

In [ ]:
file_path = 'mnt/Champagne_Sales.csv'
data = pd.read_csv(file_path, parse_dates=['date'], index_col='date')

# Renaming the sales data column to 'sales' and defining its datatype as float
data.columns = ['sales']
data['sales'] = data['sales'].astype(float)

# Delete data of 1972, since data are not available for the entire year
data = data[data.index.year != 1972]

# Show dataframe
data.head()

In [ ]:
# Cut champagne sales data for model training
data_cut = data[data.index.year < 1968]
data_cut.tail()

# Simulations of Black Box Forecast

In [ ]:
# Init pycaret setup
s = setup(
    data=data_cut,
    target='sales',
    fh=12,
    fold = 2,
    session_id=42,
)

# Create model

In [ ]:
# Create ARIMA
arima = create_model("auto_arima")

In [ ]:
# Create prophet
prophet = create_model("prophet", weekly_seasonality=True, daily_seasonality=True)

# Create Predictions

In [10]:
# Create predictions with ARIMA for 5 years
y_pred_arima = predict_model(arima, fh = 60)

In [11]:
# Create predictions with prophet for 5 years
y_pred_prophet = predict_model(prophet, fh = 60)

In [17]:
# Map sales data to the black box prediction time series
start_date = '1967-01-01'
end_date = '1971-12-01'
data_filt = data[start_date:end_date]

# Create results dataframe
result = pd.DataFrame()
result.index = data_filt.index
result['arima'] = y_pred_arima['y_pred'].values
result['prophet'] = y_pred_prophet['y_pred'].values
result['sales'] = data_filt['sales'].values
result['BlackBox'] = result[['arima', 'prophet']].mean(axis=1)
result.drop(columns=['arima', 'prophet'], inplace=True)

In [ ]:
result.head()

In [ ]:
result.plot()

In [ ]:
# Angenommen, Ihr DataFrame heißt df
path = 'mnt/BlackBox_forecast_simulation.csv'  # Pfad anpassen, wo die CSV gespeichert werden soll
result.to_csv(path)  # index=False, falls Sie den Index nicht mit speichern möchten
